In [311]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


In [312]:
data = pd.read_csv("prj-ML-model-LT_OV30/modeling_data.csv", encoding = "cp949")
print(data.head())
print(data.shape)

            dlvry_id  reg_hour  ord_price  actual_dlvry_distance  pick_floor  \
0  20230827_9376QTJN        11      22500                   1274         1.0   
1  20230827_1765MBEB        17      15700                   2809         1.0   
2  20230827_0741UTNW        18      20400                   1961        -1.0   
3  20230827_6306VAVZ        19      16900                   1826         1.0   
4  20230827_5544DEBH        11      21000                   1184         2.0   

  pick_rgn2_nm pick_rgn3_nm pick_category pick_건물용도  dlvry_지상층수  dlvry_지하층수  \
0          서초구          잠원동       CHINESE      업무시설         NaN         NaN   
1          서초구          반포동         SNACK     일반음식점        19.0         5.0   
2          서초구          서초동     ZZIM_TANG       아파트         4.0         1.0   
3          서초구          방배동         ASIAN       아파트         5.0         0.0   
4          서초구          서초동          CAFE       NaN         4.0         1.0   

  dlvry_건물용도 day_of_week  is_holiday    기온  

In [313]:
# 건물 정보 없는 데이터 삭제 
filtered_data = data.dropna(subset=['pick_건물용도', 'dlvry_건물용도'])
print(filtered_data.shape) #44,644 
print(filtered_data['outcome'].value_counts()) # 38,816, 5,828 

(44644, 19)
outcome
0    38816
1     5828
Name: count, dtype: int64


In [314]:
print(filtered_data.isna().sum())

dlvry_id                    0
reg_hour                    0
ord_price                   0
actual_dlvry_distance       0
pick_floor                  0
pick_rgn2_nm                0
pick_rgn3_nm                0
pick_category               0
pick_건물용도                   0
dlvry_지상층수               5726
dlvry_지하층수               5726
dlvry_건물용도                  0
day_of_week                 0
is_holiday                  0
기온                          0
강수량                         0
dlvry_rgn2_nm               0
dlvry_rgn3_nm               0
outcome                     0
dtype: int64


In [315]:
print(filtered_data.dtypes)

dlvry_id                  object
reg_hour                   int64
ord_price                  int64
actual_dlvry_distance      int64
pick_floor               float64
pick_rgn2_nm              object
pick_rgn3_nm              object
pick_category             object
pick_건물용도                 object
dlvry_지상층수               float64
dlvry_지하층수               float64
dlvry_건물용도                object
day_of_week               object
is_holiday                  bool
기온                       float64
강수량                      float64
dlvry_rgn2_nm             object
dlvry_rgn3_nm             object
outcome                    int64
dtype: object


In [316]:
# dlvry_지하층수 음수로
# filtered_data['dlvry_지하층수'] = pd.to_numeric(filtered_data['dlvry_지하층수'], errors='coerce')
# filtered_data['dlvry_지상층수'] = pd.to_numeric(filtered_data['dlvry_지상층수'], errors='coerce')
# filtered_data['pick_floor'] = pd.to_numeric(filtered_data['pick_floor'], errors='coerce')

filtered_data['dlvry_지하층수'] = filtered_data['dlvry_지하층수'].apply(lambda x: -x)
#print(filtered_data.head())
print(filtered_data.isna().sum())

dlvry_id                    0
reg_hour                    0
ord_price                   0
actual_dlvry_distance       0
pick_floor                  0
pick_rgn2_nm                0
pick_rgn3_nm                0
pick_category               0
pick_건물용도                   0
dlvry_지상층수               5726
dlvry_지하층수               5726
dlvry_건물용도                  0
day_of_week                 0
is_holiday                  0
기온                          0
강수량                         0
dlvry_rgn2_nm               0
dlvry_rgn3_nm               0
outcome                     0
dtype: int64


## 1. 데이터 전처리


## 1-1. pick_건물용도/ dlvry_건물용도 : 개수 축소

In [317]:
print(filtered_data['pick_건물용도'].value_counts())

pick_건물용도
업무시설          14372
근린생활시설         7875
아파트            5367
일반음식점          5206
일반상점           3881
공동주택           3089
교육시설           1604
의료시설           1177
백화점             534
자동차관련시설         471
오피스텔            266
기타시설            210
숙박시설            174
대형판매점           141
위락시설            134
문화시설             87
종교시설             30
체육시설             16
위험물저장및처리시설       10
Name: count, dtype: int64


In [318]:
# 데이터 정리
values_to_replace = ['위락시설', '문화시설', '종교시설', '체육시설','위험물저장및처리시설']
filtered_data['pick_건물용도'] = data['pick_건물용도'].replace(values_to_replace, '기타시설')
print(filtered_data['pick_건물용도'].value_counts())

pick_건물용도
업무시설       14372
근린생활시설      7875
아파트         5367
일반음식점       5206
일반상점        3881
공동주택        3089
교육시설        1604
의료시설        1177
백화점          534
기타시설         487
자동차관련시설      471
오피스텔         266
숙박시설         174
대형판매점        141
Name: count, dtype: int64


In [319]:
print(filtered_data['dlvry_건물용도'].value_counts())

dlvry_건물용도
공동주택          16952
아파트           13910
업무시설           7279
근린생활시설         1559
일반상점           1224
교육시설           1028
의료시설            758
일반음식점           557
숙박시설            459
공공시설            241
종교시설            127
교육연구시설          120
오피스텔            101
자동차관련시설          67
방송통신시설           53
기타시설             39
체육시설             36
백화점              36
위험물저장및처리시설       18
복지시설             16
문화시설             14
위락시설             12
복합몰               7
창고시설              6
수련시설              6
동식물시설             5
단독주택              5
운수시설              3
노유자시설             2
호텔                2
체력시설              1
공공용시설             1
Name: count, dtype: int64


In [320]:
value_counts = filtered_data['dlvry_건물용도'].value_counts()

threshold = 100
filtered_data['dlvry_건물용도'] = filtered_data['dlvry_건물용도'].apply(lambda x: '기타시설' if value_counts[x] <= threshold else x)

filtered_data['dlvry_건물용도'] = filtered_data['dlvry_건물용도'].replace('교육연구시설', '교육시설')
filtered_data['dlvry_건물용도'] = filtered_data['dlvry_건물용도'].replace('공공용시설', '공공시설')

print(filtered_data['dlvry_건물용도'].value_counts())

dlvry_건물용도
공동주택      16952
아파트       13910
업무시설       7279
근린생활시설     1559
일반상점       1224
교육시설       1148
의료시설        758
일반음식점       557
숙박시설        459
기타시설        329
공공시설        241
종교시설        127
오피스텔        101
Name: count, dtype: int64


## 1-2. dlvry_지상층수 /지하층수 : dlvry_건물용도 median 값 가져오기

In [321]:
median_values = filtered_data.dropna().groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].median()
mean_values = filtered_data.dropna().groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].mean()

print("Median Values:")
print(median_values)

print("\nMean Values:")
print(mean_values)


Median Values:
            dlvry_지상층수  dlvry_지하층수
dlvry_건물용도                        
공공시설               3.0        -0.0
공동주택               4.0        -1.0
교육시설               6.0        -1.0
근린생활시설             5.0        -1.0
기타시설               5.0        -2.0
숙박시설              10.0        -1.0
아파트               15.0        -1.0
업무시설              10.0        -3.0
오피스텔              15.0        -7.0
의료시설              10.0        -1.0
일반상점               4.0        -1.0
일반음식점              5.0        -1.0
종교시설               4.0        -3.0

Mean Values:
            dlvry_지상층수  dlvry_지하층수
dlvry_건물용도                        
공공시설          3.066390   -0.203320
공동주택          3.844208   -0.732086
교육시설          6.414397   -1.383268
근린생활시설        5.316228   -1.250160
기타시설          8.412698   -2.501587
숙박시설         10.838428   -2.397380
아파트          18.313515   -1.368009
업무시설         11.697593   -3.120231
오피스텔         15.000000   -6.118812
의료시설          9.507916   -1.928760
일반상점          5.041667   -

In [322]:
# na 값을 중앙값으로 채우기
filtered_data['dlvry_지상층수'].fillna(filtered_data['dlvry_건물용도'].map(median_values['dlvry_지상층수']), inplace=True)
filtered_data['dlvry_지하층수'].fillna(filtered_data['dlvry_건물용도'].map(median_values['dlvry_지하층수']), inplace=True)

print(filtered_data.describe())
print(filtered_data.columns)

           reg_hour      ord_price  actual_dlvry_distance    pick_floor  \
count  44644.000000   44644.000000           44644.000000  44644.000000   
mean      15.780866   27080.946757            1959.281538      0.748006   
std        3.745221   17994.351039            1156.483892      0.924460   
min        9.000000    5000.000000               1.000000     -1.000000   
25%       12.000000   16700.000000            1115.000000      1.000000   
50%       16.000000   22400.000000            1756.000000      1.000000   
75%       19.000000   31400.000000            2514.000000      1.000000   
max       23.000000  483000.000000            8003.000000      8.000000   

         dlvry_지상층수    dlvry_지하층수            기온           강수량       outcome  
count  44644.000000  44644.000000  44644.000000  44644.000000  44644.000000  
mean      10.028380     -1.476481     26.778833      0.609437      0.130544  
std        8.766227      1.608017      2.664765      3.162902      0.336905  
min        0

In [323]:
print(filtered_data.isna().sum())

dlvry_id                 0
reg_hour                 0
ord_price                0
actual_dlvry_distance    0
pick_floor               0
pick_rgn2_nm             0
pick_rgn3_nm             0
pick_category            0
pick_건물용도                0
dlvry_지상층수               0
dlvry_지하층수               0
dlvry_건물용도               0
day_of_week              0
is_holiday               0
기온                       0
강수량                      0
dlvry_rgn2_nm            0
dlvry_rgn3_nm            0
outcome                  0
dtype: int64


## 1-3.  pick_floor, dlvry_지상층수, dlvry_지하층수 

In [324]:
print(filtered_data['pick_floor'].value_counts())

pick_floor
 1.0    31450
-1.0     7548
 2.0     3810
 0.0     1363
 3.0      314
 4.0       85
 8.0       73
 6.0        1
Name: count, dtype: int64


In [325]:
print(filtered_data['dlvry_지하층수'].value_counts())

dlvry_지하층수
-1.0    24736
-0.0     9058
-2.0     3590
-3.0     1876
-4.0     1778
-5.0     1483
-6.0     1233
-7.0      674
-8.0      215
-9.0        1
Name: count, dtype: int64


In [326]:
print(filtered_data['dlvry_지상층수'].value_counts())
print(filtered_data['dlvry_지상층수'].describe())

dlvry_지상층수
4.0     9546
5.0     5935
3.0     4366
15.0    4286
6.0     2892
2.0     1796
12.0    1556
7.0     1408
20.0    1372
10.0    1270
13.0    1182
29.0     872
32.0     782
14.0     722
9.0      713
25.0     599
18.0     551
24.0     526
8.0      522
11.0     466
27.0     413
22.0     412
16.0     329
17.0     269
37.0     267
21.0     262
19.0     248
46.0     184
26.0     177
1.0      171
23.0     166
31.0     114
66.0      52
28.0      43
69.0      40
55.0      30
51.0      26
34.0      22
44.0      20
45.0      16
0.0        9
30.0       8
33.0       4
Name: count, dtype: int64
count    44644.000000
mean        10.028380
std          8.766227
min          0.000000
25%          4.000000
50%          6.000000
75%         15.000000
max         69.000000
Name: dlvry_지상층수, dtype: float64


In [327]:
def categorize_floor(value):
    if value <= 0:
        return '지하'
    elif 1 <= value <= 5:
        return '저층'
    elif 6 <= value <= 10 :
        return '중층'
    elif 11 <= value <= 15 : 
        return '중상층'
    elif 16 <= value <= 29 :
        return '고층'
    elif 30 <= value <= 49 :
        return '준초고층'
    elif 50 <= value:
        return '초고층'

filtered_data['pick_floor'] = filtered_data['pick_floor'].apply(categorize_floor)
filtered_data['dlvry_지상층수'] = filtered_data['dlvry_지상층수'].apply(categorize_floor)

print(filtered_data['dlvry_지상층수'].value_counts())
print(filtered_data['pick_floor'].value_counts())

dlvry_지상층수
저층      21814
중상층      8212
중층       6805
고층       6239
준초고층     1417
초고층       148
지하          9
Name: count, dtype: int64
pick_floor
저층    35659
지하     8911
중층       74
Name: count, dtype: int64


## 2. 데이터 전처리

In [328]:
print(filtered_data.shape)
print(filtered_data.info())

(44644, 19)
<class 'pandas.core.frame.DataFrame'>
Index: 44644 entries, 1 to 68026
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dlvry_id               44644 non-null  object 
 1   reg_hour               44644 non-null  int64  
 2   ord_price              44644 non-null  int64  
 3   actual_dlvry_distance  44644 non-null  int64  
 4   pick_floor             44644 non-null  object 
 5   pick_rgn2_nm           44644 non-null  object 
 6   pick_rgn3_nm           44644 non-null  object 
 7   pick_category          44644 non-null  object 
 8   pick_건물용도              44644 non-null  object 
 9   dlvry_지상층수             44644 non-null  object 
 10  dlvry_지하층수             44644 non-null  float64
 11  dlvry_건물용도             44644 non-null  object 
 12  day_of_week            44644 non-null  object 
 13  is_holiday             44644 non-null  bool   
 14  기온                     44644 non-null  float64


In [329]:
dataset = filtered_data[['reg_hour', 'ord_price','actual_dlvry_distance', 'pick_floor', 'pick_rgn3_nm', 
                         'pick_category', 'pick_건물용도','dlvry_지상층수', 'dlvry_지하층수',
                         'dlvry_건물용도','day_of_week', 'is_holiday', '기온', '강수량', 'dlvry_rgn2_nm', 'dlvry_rgn3_nm', 'outcome'
                        ]]
dataset.isna().sum()
dataset.to_csv('prj-ML-model-LT_OV30/modeling_data_final.csv', index= False, encoding = "utf-8")

## 2-1. factor 변수 / one-hot-encoding

In [330]:
# category  
for col in ['reg_hour','pick_floor', 'pick_rgn3_nm', 'pick_category', 'pick_건물용도','dlvry_지상층수','dlvry_건물용도',
            'day_of_week', 'is_holiday','dlvry_rgn2_nm', 'dlvry_rgn3_nm', 'outcome' ] : 
    dataset[col] = dataset[col].astype('category')

print(dataset.dtypes)

reg_hour                 category
ord_price                   int64
actual_dlvry_distance       int64
pick_floor               category
pick_rgn3_nm             category
pick_category            category
pick_건물용도                category
dlvry_지상층수               category
dlvry_지하층수                float64
dlvry_건물용도               category
day_of_week              category
is_holiday               category
기온                        float64
강수량                       float64
dlvry_rgn2_nm            category
dlvry_rgn3_nm            category
outcome                  category
dtype: object


In [331]:
var = ['reg_hour','pick_floor', 'pick_rgn3_nm', 'pick_category', 'pick_건물용도','dlvry_지상층수','dlvry_건물용도',
            'day_of_week', 'is_holiday','dlvry_rgn2_nm', 'dlvry_rgn3_nm' ] 
encoder = OneHotEncoder()
onehot = pd.DataFrame(encoder.fit_transform(dataset[var]).toarray(), columns=encoder.get_feature_names_out(var), index = dataset.index)
dataset = pd.concat([onehot, dataset.drop(columns=var)], axis=1)

print(dataset.columns)

Index(['reg_hour_9', 'reg_hour_10', 'reg_hour_11', 'reg_hour_12',
       'reg_hour_13', 'reg_hour_14', 'reg_hour_15', 'reg_hour_16',
       'reg_hour_17', 'reg_hour_18',
       ...
       'dlvry_rgn3_nm_잠원동', 'dlvry_rgn3_nm_주암동', 'dlvry_rgn3_nm_청담동',
       'dlvry_rgn3_nm_흑석동', 'ord_price', 'actual_dlvry_distance', 'dlvry_지하층수',
       '기온', '강수량', 'outcome'],
      dtype='object', length=124)


## 2-2 numeric variables

In [332]:
scaler = StandardScaler()  #평균 0 , 분산 1로 조정
#scaler = MinMaxScaler()

num_vars = ['ord_price', 'actual_dlvry_distance', 'dlvry_지하층수', '기온', '강수량']
dataset[num_vars] = scaler.fit_transform(dataset[num_vars])

print(dataset.head(3))

   reg_hour_9  reg_hour_10  reg_hour_11  reg_hour_12  reg_hour_13  \
1         0.0          0.0          0.0          0.0          0.0   
2         0.0          0.0          0.0          0.0          0.0   
3         0.0          0.0          0.0          0.0          0.0   

   reg_hour_14  reg_hour_15  reg_hour_16  reg_hour_17  reg_hour_18  ...  \
1          0.0          0.0          0.0          1.0          0.0  ...   
2          0.0          0.0          0.0          0.0          1.0  ...   
3          0.0          0.0          0.0          0.0          0.0  ...   

   dlvry_rgn3_nm_잠원동  dlvry_rgn3_nm_주암동  dlvry_rgn3_nm_청담동  dlvry_rgn3_nm_흑석동  \
1                0.0                0.0                0.0                0.0   
2                0.0                0.0                0.0                0.0   
3                0.0                0.0                0.0                0.0   

   ord_price  actual_dlvry_distance  dlvry_지하층수        기온       강수량  outcome  
1  -0.632480      

## 3. train/test set split

In [333]:
# set.seed(1234)
# train_ratio = 0.75
# total_samples = dataset.shape[0]
# train_samples = int(train_ratio * total_samples)
# train_set = dataset[:train_samples]
# test_set = dataset[train_samples:]

# print(train_set.shape) #33483
# print(test_set.shape) #11161

In [334]:
outcome_counts = dataset['outcome'].value_counts()
train_set, test_set = train_test_split(dataset, test_size=0.25, stratify = dataset['outcome'], random_state=1234)

train_outcome_counts = train_set['outcome'].value_counts()
test_outcome_counts = test_set['outcome'].value_counts()

print(train_outcome_counts)
print(test_outcome_counts)

print(train_set.shape) #33483
print(test_set.shape) #11161

outcome
0    29112
1     4371
Name: count, dtype: int64
outcome
0    9704
1    1457
Name: count, dtype: int64
(33483, 124)
(11161, 124)


In [335]:
X_train = train_set.drop(columns=['outcome'])
y_train = train_set['outcome']

X_test = test_set.drop(columns=['outcome'])
y_test = test_set['outcome']

print(X_train.columns)

Index(['reg_hour_9', 'reg_hour_10', 'reg_hour_11', 'reg_hour_12',
       'reg_hour_13', 'reg_hour_14', 'reg_hour_15', 'reg_hour_16',
       'reg_hour_17', 'reg_hour_18',
       ...
       'dlvry_rgn3_nm_자곡동', 'dlvry_rgn3_nm_잠원동', 'dlvry_rgn3_nm_주암동',
       'dlvry_rgn3_nm_청담동', 'dlvry_rgn3_nm_흑석동', 'ord_price',
       'actual_dlvry_distance', 'dlvry_지하층수', '기온', '강수량'],
      dtype='object', length=123)


## 4. modeling - Logistic Regression

In [ ]:
X_train = sm.add_constant(X_train) 
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

print(result.summary())

X_test = sm.add_constant(X_test) 
y_pred = result.predict(X_test)  

y_pred_binary = (y_pred > 0.5).astype(int)

conf_matrix = confusion_matrix(y_test, y_pred_binary)
accuracy = accuracy_score(y_test, y_pred_binary)
sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

print("\nConfusion Matrix:")
print(conf_matrix)
print("\nAccuracy:", accuracy)
print("Sensitivity (True Positive Rate):", sensitivity)
print("Specificity (True Negative Rate):", specificity)


In [337]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(data):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return vif_data

# 다중 공선성 확인
vif_result = calculate_vif(X_train)
print(vif_result)


LinAlgError: SVD did not converge

In [ ]:
glm_2 = LogisticRegression(penalty='l2')
glm_2.fit(X_train, y_train)

variable_names = X_train.columns
coef = glm_2.coef_[0]

coef_table = pd.DataFrame({'Variable': variable_names, 'Coefficient': coef})

logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
p_values = result.pvalues
coef_table['p-value'] = p_values

print(coef_table)

         Current function value: 0.303403
         Iterations: 35
                  Variable  Coefficient  p-value
0                    const    -0.165094      NaN
1               reg_hour_9    -0.074391      NaN
2              reg_hour_10    -0.683044      NaN
3              reg_hour_11    -1.232871      NaN
4              reg_hour_12    -0.994115      NaN
..                     ...          ...      ...
119              ord_price     0.121922      NaN
120  actual_dlvry_distance     0.669519      NaN
121             dlvry_지하층수     0.026519      NaN
122                     기온    -0.178313      NaN
123                    강수량     0.007233      NaN

[124 rows x 3 columns]


In [ ]:
pred_glm = glm_2.predict(X_test)
class_report = classification_report(y_test, pred_glm)
print(class_report)

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      9781
           1       0.48      0.24      0.32      1380

    accuracy                           0.87     11161
   macro avg       0.69      0.60      0.63     11161
weighted avg       0.85      0.87      0.85     11161

